In [1]:
import leuvenmapmatching as mm
from leuvenmapmatching.map.inmem import InMemMap
from leuvenmapmatching.matcher.distance import DistanceMatcher
import pickle
import json
import numpy as np
import networkx as nx
import osmnx as ox
import tqdm, os
from collections import Counter, defaultdict

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
data_file = map_file = "../dataset/"
# G = json.load(open(map_file + "/map.json", 'r'))
f = open(data_file + "longhua_4k_sim.pkl", "rb")
G = pickle.load(f)
for node in G.nodes:
    x, y = G.nodes[node]['x'], G.nodes[node]['y']
    G.nodes[node]['xy'] = [x, y]

In [4]:
import re
import json
from collections import OrderedDict
import osmnx as ox
# 读取文本文件中的内容
with open(
        "../dataset/sim_traj/trajectries.txt",
        "r") as f:
    content = f.read()

# 使用正则表达式匹配每个 vehicle 的经纬度信息
pattern = r"vehicle_(\d+):(.+)"
matches = re.findall(pattern, content)

# 将匹配到的经纬度信息转换成字典格式，并保存为 JSON 文件
data = {}
for match in matches:
    vehicle_id, trajectory_str = match[0], match[1]
    trajectory_list = []
    for point_str in trajectory_str.split(","):
        lon_match = re.search(r"lon:(\d+\.\d+)", point_str)
        lat_match = re.search(r"lat:(\d+\.\d+)", point_str)
        if lon_match and lat_match:
            lon, lat = float(lon_match.group(1)), float(lat_match.group(1))
            trajectory_list.append([lon, lat])
    ans = list(OrderedDict.fromkeys(tuple(x) for x in trajectory_list))[::16]
    data[int(vehicle_id)] = ans
print(len(data))

4758


In [5]:
tklet_path = '../dataset/sim_traj/tracklet.json'
import json

tklet = json.load(open(tklet_path, 'rb'))

In [6]:
with open(data_file + '/traj_nodes_4k.pkl', 'rb') as f:
    traj_nodes = pickle.load(f)
traj_veh = defaultdict(list)
for k in traj_nodes:
    traj_veh[k['vehicle_id']] = np.array(
        k['xyt'])[:, :1].astype(int).reshape(-1).tolist()

In [7]:
f = open(data_file + "longhua_1.8k.pkl", "rb")
G2 = pickle.load(f)
for node in G2.nodes:
    x, y = G2.nodes[node]['x'], G2.nodes[node]['y']
    G2.nodes[node]['xy'] = [x, y]
map_oid2nid = defaultdict()
for n in G2.nodes():
    if isinstance(G2.nodes()[n]['osmid_original'], int):
        map_oid2nid[G2.nodes()[n]['osmid_original']] = n
        continue
    osmid = eval(G2.nodes()[n]['osmid_original'])
    for oid in osmid:
        map_oid2nid[oid] = n
len(map_oid2nid.keys())

3931

In [8]:
for ky, vlu in traj_veh.items():
    route = []
    for n in traj_veh[ky]:
        if n in map_oid2nid.keys():
            route.append(map_oid2nid[n])
    traj_veh[ky] = [
        x for i, x in enumerate(route) if i == 0 or x != route[i - 1]
    ]


In [9]:
fig_tklet2node = defaultdict(list)
cnt = 0
cnt_true = 0
camid2nodeid = {}
for fig_id in tqdm.tqdm(tklet.keys()):
    lon_match = re.search(r"lon:(\d+\.\d+)", tklet[fig_id]['camera_loc'])
    lat_match = re.search(r"lat:(\d+\.\d+)", tklet[fig_id]['camera_loc'])
    cam_id = tklet[fig_id]['camera_id'][11:]
    vid = int(tklet[fig_id]['vehicle_id'].split('_')[1])
    # print(traj_veh[vid], paths)
    # if vid not in traj_veh.keys():
    #     continue
    # paths = traj_veh[vid]
    # if cam_id in camid2nodeid.keys():
    #     cam_node1 = camid2nodeid[cam_id]
    # else:
    #     ans = float(lon_match.group(1)), float(lat_match.group(1))
    #     cam_node1 = ox.nearest_nodes(G2.subgraph(paths), *ans)
    #     camid2nodeid[cam_id] = cam_node1
    ans = float(lon_match.group(1)), float(lat_match.group(1))
    cam_node = ox.nearest_nodes(G2, *ans)
    # # cam_node1 = ox.nearest_nodes(G.subgraph(traj_veh[vid]), *ans)
    # if cam_node != cam_node1:
    #     # print(cam_node, cam_node1)
    #     # print(cam_node)
    #     cnt += 1
    # else:
    #     cnt_true += 1
    tklet_node = [cam_node]
    fig_tklet2node[str(fig_id) + "_" + str(cam_node)] = tklet_node
    tklet_list = []
    for point_str in tklet[fig_id]['tracklet'].split(","):
        lon_match = re.search(r"lon:(\d+\.\d+)", point_str)
        lat_match = re.search(r"lat:(\d+\.\d+)", point_str)
        if lon_match and lat_match:
            lon, lat = float(lon_match.group(1)), float(lat_match.group(1))
            tklet_list.append([lon, lat])
    # print(tklet_list)
    if len(tklet_list) < 2:
        # print(fig_id)
        continue
    unique_lst = list(OrderedDict.fromkeys(tuple(x) for x in tklet_list))
    bearing_from = ox.bearing.calculate_bearing(unique_lst[1][1],
                                                unique_lst[1][0],
                                                unique_lst[0][1],
                                                unique_lst[0][0])
    bearing_to = ox.bearing.calculate_bearing(unique_lst[-2][1],
                                              unique_lst[-2][0],
                                              unique_lst[-1][1],
                                              unique_lst[-1][0])
    for nbs in list(G2.predecessors(cam_node)):
        node_bearing = ox.bearing.calculate_bearing(
            *G2.nodes()[cam_node]['xy'][::-1],
            *G2.nodes()[nbs]['xy'][::-1])
        if abs(node_bearing - bearing_from) > 180:
            if node_bearing < bearing_from:
                node_bearing += 360
            else:
                bearing_from += 360
        if abs(node_bearing - bearing_from) < 20 and tklet_node[0] == cam_node:
            tklet_node.insert(0, nbs)
    for nbs in list(G2.successors(cam_node)):
        node_bearing = ox.bearing.calculate_bearing(
            *G2.nodes()[cam_node]['xy'][::-1],
            *G2.nodes()[nbs]['xy'][::-1])
        if abs(node_bearing - bearing_to) > 180:
            if node_bearing < bearing_to:
                node_bearing += 360
            else:
                bearing_to += 360
        if abs(node_bearing - bearing_to) < 20 and tklet_node[-1] == cam_node:
            tklet_node.append(nbs)
    if len(tklet_node) > 3:
        tklet_node.pop(1)
print(cnt, cnt_true)

100%|██████████| 44307/44307 [06:43<00:00, 109.93it/s]

0 0


In [22]:
fig_names = defaultdict(list)
for fig_id in fig_tklet2node.keys():
    img_id = fig_id.split('_')[0]
    fig_names[tklet[img_id]['image_name'][:-4]] = fig_tklet2node[fig_id]
# with open(data_file + '/tklet_in_figs.pkl', 'wb') as f:
#     pickle.dump(fig_names, f)

In [23]:
import pandas as pd

df_recd = pd.read_pickle(
    open('../dataset/records_4w.pkl', 'rb'))
df_recd = pd.DataFrame.from_dict(df_recd, orient='columns')
# df_recd['time'] = df_recd['time']

In [24]:
df_recd.head(3)

,vehicle_id,time,image_name,camera_id,car_feature,tklet,plate_feature,plate_text
0,11026,468.644714,image_585_0716_01.jpg,101,"[-0.05570761, 0.011447539, -0.0491364, -0.0229...","[[114.019806, 22.634327], [114.01981, 22.63433...",None,None
1,11026,467.684509,image_580_0716_01.jpg,541,"[-0.034275193, 0.0321603, -0.08746838, -0.0010...","[[114.017185, 22.631675], [114.017196, 22.6316...",None,None
2,11026,463.80368,image_566_0716_01.jpg,530,"[-0.037081357, -0.025386535, -0.0337778, -0.01...","[[114.008546, 22.62041], [114.008547, 22.62042...",None,None


In [25]:
df_recd['image_name'].values

array(['image_585_0716_01.jpg', 'image_580_0716_01.jpg',
       'image_566_0716_01.jpg', ..., 'image_4954_0707_004.jpg',
       'image_4898_0707_004.jpg', 'image_4956_0707_004.jpg'], dtype=object)

In [26]:
with open(data_file + '/camid2nodeid.pkl', 'rb') as f:
    camid2nodeid = pickle.load(f)

In [27]:
df_recd['tklet_nodes'] = df_recd['image_name'].apply(
    lambda x: fig_names[x[:-4]])
df_recd.drop(columns=['tklet'], inplace=True)
df_recd['camera_id'] = df_recd['camera_id'].apply(lambda x: camid2nodeid[x])

In [29]:
pickle.dump(df_recd.to_dict('records'), open('records_4w_tklet.pkl', 'wb'))

In [30]:
def find_longest_common_continuous_substring(str1, str2):
    m = len(str1)
    n = len(str2)

    # 创建一个二维数组来保存子问题的结果
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    max_length = 0  # 最长公共连续子串的长度
    end = 0  # 最长公共连续子串的结束位置

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                if dp[i][j] > max_length:
                    max_length = dp[i][j]
                    end = i  # 记录最长公共连续子串的结束位置

    longest_common_continuous_substring = str1[end - max_length:end]

    return longest_common_continuous_substring

In [31]:
veh_tklet = defaultdict(list)
tklet_pre = []
tklet_iou = []
cnt = 0
cnt_grth = 0
for fig_id in fig_tklet2node.keys():
    img_id = fig_id.split('_')[0]
    vid = int(tklet[img_id]['vehicle_id'].split('_')[1])
    ans = camid2nodeid[int(tklet[img_id]['camera_id'][11:])]
    if not vid in traj_veh.keys():
        # print(vid)
        continue
    if ans in traj_veh[int(vid)]:
        intn = find_longest_common_continuous_substring(
            traj_veh[int(vid)], fig_tklet2node[fig_id])
        node_ix = traj_veh[int(vid)].index(ans)

        grth_tklet = traj_veh[int(vid)][max(0, node_ix - 1):node_ix + 2]
        tklet_iou.append((len(intn)) / (len(grth_tklet)))
        tklet_pre.append((len(set(fig_tklet2node[fig_id]) & set(intn))) /
                         (len(set(fig_tklet2node[fig_id]))))
        cnt_grth += 1
    else:
        cnt += 1
        # tklet_pre.append(0)
    # veh_tklet[tklet[img_id]['vehicle_id'].split('_')[1]] = fig_tklet2node[fig_id]
print(sum(tklet_iou) / len(tklet_iou))
print(sum(tklet_pre) / len(tklet_pre))

0.8783145572289169
0.9362235122811841
